# Тестовое задание

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

In [58]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear')

## Cкачиваем файл с данными

In [59]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [60]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [61]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [62]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [94]:
dat_t, dat = dat, dat[0:50001]

In [95]:
train_dat, test_dat, train_ans, test_ans = train_test_split(
    dat.reviewText, dat.bin, test_size=0.2)

## Натравим на необработанные данные

In [96]:
count_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
tfidf_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
count_train_mtrx_dft = count_vect.transform(train_dat.apply(lambda x: np.str_(x)))
tfidf_train_mtrx_dft = tfidf_vect.transform(train_dat.apply(lambda x: np.str_(x)))
count_test_mtrx_dft = count_vect.transform(test_dat.apply(lambda x: np.str_(x)))
tfidf_test_mtrx_dft = tfidf_vect.transform(test_dat.apply(lambda x: np.str_(x)))

## Приводим к базовой форме

In [97]:
train_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), train_dat.apply(lambda x: np.str_(x)))))
test_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), test_dat.apply(lambda x: np.str_(x)))))
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [98]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [99]:
count_vect.fit(dat.words)
tfidf_vect.fit(dat.words)
count_train_mtrx = count_vect.transform(train_dat_tr)
tfidf_train_mtrx = tfidf_vect.transform(train_dat_tr)
count_test_mtrx = count_vect.transform(test_dat_tr)
tfidf_test_mtrx = tfidf_vect.transform(test_dat_tr)

## Проверим, что получилось

In [100]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
4996,B00001WRSJ,Pro:Great sound. Fairly effective sound isola...,3,False,pro great sound fairli effect sound isol well ...
4997,B00001WRSJ,I did not know that Sony made headphones of th...,5,True,did not know that soni made headphon of thi qu...
4998,B00001WRSJ,"Just like the title of my review says, I've ha...",5,True,just like the titl of my review say ve had the...
4999,B00001WRSJ,I love these headphones. The sound is superb. ...,4,True,love these headphon the sound is superb bass t...


## Разобьем вектора на выборки

In [101]:
# count_train_mtrx, count_test_mtrx, count_train_ans, count_test_ans = train_test_split(
#     dat_count, dat.bin, test_size=0.2)
# #dat_count.toarray()

In [102]:
# tfidf_train_mtrx, tfidf_test_mtrx, tfidf_train_ans, tfidf_test_ans = train_test_split(
#     dat_tfidf, dat.bin, test_size=0.2)
# #dat_tfidf.toarray()

In [103]:
# count_train_mtrx_dft, count_test_mtrx_dft, count_train_ans_dft, count_test_ans_dft = train_test_split(
#     dat_count_deft, dat.bin, test_size=0.2)

In [104]:
# tfidf_train_mtrx_dft, tfidf_test_mtrx_dft, tfidf_train_ans_dft, tfidf_test_ans_dft = train_test_split(
#     dat_tfidf_deft, dat.bin, test_size=0.2)

# Обучим модели и посмотрим их точность

## Обработанные

In [105]:
clf.fit(count_train_mtrx, train_ans)
cout_pred = clf.predict(count_test_mtrx)
accuracy_score(cout_pred, test_ans)

0.8441558441558441

In [106]:
clf.fit(tfidf_train_mtrx, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx)
accuracy_score(tfidf_pred, test_ans)

0.8221778221778222

## Дефолтные

In [107]:
clf.fit(count_train_mtrx_dft, train_ans)
cout_pred = clf.predict(count_test_mtrx_dft)
accuracy_score(cout_pred, test_ans)

0.8471528471528471

In [108]:
clf.fit(tfidf_train_mtrx_dft, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx_dft)
accuracy_score(tfidf_pred, test_ans)

0.8161838161838162

## Обработанные с оптимальными параметрами

In [109]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx, train_ans)
print(search.best_params_)

{'C': 0.30000000000000004, 'penalty': 'l2'}


In [110]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx))

0.8391608391608392

In [111]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx,  train_ans)
print(search.best_params_)

{'C': 2.6, 'penalty': 'l1'}


In [112]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx))

0.8521478521478522

## Дефолтные с оптимальными параметрами

In [113]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 0.2, 'penalty': 'l2'}


In [114]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx_dft))

0.8491508491508492

In [115]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 2.9000000000000004, 'penalty': 'l1'}


In [116]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft))

0.8421578421578422

## Проверим баланс классов

In [117]:
print("All :\n1 - ", sum(dat.bin), "\n0 - ", len(dat.bin) - sum(dat.bin)) 
print("Train :\n1 - ", sum(train_ans), "\n0 - ", len(train_ans) - sum(train_ans)) 
print("Test :\n1 - ", sum(test_ans), "\n0 - ", len(test_ans) - sum(test_ans)) 

All :
1 -  4036 
0 -  965
Train :
1 -  3234 
0 -  766
Test :
1 -  802 
0 -  199
